In [1]:
from datetime import datetime, timedelta
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm
import time
import psycopg2
import warnings
import calendar

In [2]:
# Suppress all warnings globally
warnings.simplefilter(action='ignore', category=UserWarning)

In [3]:
def connect_to_pgsql():
    """Used to connect to PostgreSQL database"""
    
    # Read PostgreSQL connection parameters from text
    with open("pgsql_credentials.txt", "r") as file:
        lines_pgsql = [line.strip() for line in file.readlines()]

    host, port, username, password = lines_pgsql[0], lines_pgsql[1], lines_pgsql[2], lines_pgsql[3]

    # Establish PostgreSQL connection
    try:
        connection = psycopg2.connect(database="mm_commodity",
                                     host=host,
                                     port=port,
                                     user=username,
                                     password=password)
        print("PostgreSQL Connection successful!")
    except psycopg2.Error as e:
        print("PostgreSQL Connection failed!", e)

    return connection

# Extract Data

In [4]:
def connect_to_website(url):
    # Send a GET request to the URL
    try:
        response = requests.get(url)
        # Check if the request was successful
        if response.status_code == 200:
            # Parse the HTML content using BeautifulSoup
            bsObj = BeautifulSoup(response.text, 'html.parser')
            return bsObj
        else:
            print(f"Failed to retrieve webpage. Status code: {response.status_code}")
            return None
    except Exception as e:
        print(f"Failed to retrieve webpage. Error: {e}")
        return None

In [5]:
def extract_data(bsObj, current_date):
    """Find commodity data by market tags"""
    
    try:

        # Find main data tag
        main_data_tag = bsObj.find('div', class_='css-1972jml')

        out_df = pd.DataFrame()
        for dummy_tag in main_data_tag:

            try:
                #extract market name
                price_data_tag_list = dummy_tag.find_all('div', class_='css-0')
                market_name = price_data_tag_list[0].text.strip()
            except IndexError:
                continue
            #extract commodity and unit of measure
            crops_tag_list = dummy_tag.find_all('p')

            crop_name_list = []
            unit_list = []

            crop_count = 1
            unit_count = 1
            for crop in range(int(len(crops_tag_list)/2)):
                crop_name = crops_tag_list[crop_count-1].text.strip()
                unit = crops_tag_list[unit_count].text.strip()

                crop_name_list.append(crop_name)
                unit_list.append(unit)

                #print(crop_name)
                #print(unit)

                crop_count += 2
                unit_count += 2

            #extract price
            price_tag_list = dummy_tag.find_all('h5')

            price_list = []
            for i in range(len(price_tag_list)):
                price = price_tag_list[i].text.strip()

                price_list.append(price)


            base_df = pd.DataFrame({"Commodity Name":crop_name_list, "Unit of Measure":unit_list, "Price":price_list})
            base_df["Market Name"] = market_name
            base_df["Date"] = current_date
            base_df = base_df[['Date','Market Name','Commodity Name','Unit of Measure','Price']]

            # split price column for max price and min price
            base_df[['Max Price', 'Min Price']] = base_df['Price'].str.split('-', expand=True)

            # replace , & ကျပ်
            base_df['Max Price'] = base_df['Max Price'].str.replace(',', '')
            base_df['Min Price'] = base_df['Min Price'].replace({',':'',
                                                                 ' ကျပ်':''}, regex=True)

            # Remove all Unicode characters from object columns
            # Remove non-printable characters from the 'Market' and 'Crop Name' columns while keeping readable text
            base_df['Market Name'] = base_df['Market Name'].str.replace(r'[\x00-\x1F\x7F\u200c\u200d\u200b]', '', regex=True)
            base_df['Commodity Name'] = base_df['Commodity Name'].str.replace(r'[\x00-\x1F\x7F\u200c\u200d\u200b]', '', regex=True)
            base_df['Unit of Measure'] = base_df['Unit of Measure'].str.replace(r'[\x00-\x1F\x7F\u200c\u200d\u200b]', '', regex=True)

            out_df = pd.concat([out_df, base_df])
        return out_df
    except Exception as e:
        print(f"An error occurred while extracting the data. Error: {e}")

In [6]:
def load_data_to_postgresql(out_df, current_date):
    if out_df is None:
        print(f"There is no data for this date: {current_date}")
    else:
        out_df.to_excel("Crop Data " + current_date + ".xlsx", index=False)
        # Step 1: Transform & Load the dimension tables
        fact_df = out_df

        # Extract unique values for two dimension columns
        fact_market_list = fact_df['Market Name'].unique().tolist()
        fact_commo_list = fact_df['Commodity Name'].unique().tolist()

        # Insert DimMarket Table

        new_market_values = [(market,) for market in fact_market_list]

        try:
            # Connect to the PostgreSQL database
            conn = connect_to_pgsql()
            print("Inserting Market Names....")
            cursor = conn.cursor()

            # SQL query to insert a value
            query = """INSERT INTO "DimMarket" (market_name) VALUES (%s)
                       ON CONFLICT (market_name) DO NOTHING;;"""

            # Execute the query for multiple rows
            cursor.executemany(query, new_market_values)

            # Commit the transaction
            conn.commit()

            print("New markets inserted successfully.")

        except psycopg2.Error as e:
            print(f"An error occurred: {e}")

        finally:
            # Close the cursor and connection
            if cursor:
                cursor.close()
            if conn:
                conn.close()

        # Update Commo Master Table and DimCommodity Table

        new_commo_values = [(commo,) for commo in fact_commo_list]

        try:
            # Connect to the PostgreSQL database
            conn = connect_to_pgsql()
            print("Inserting Commodity Names.....")
            cursor = conn.cursor()

            # SQL query to insert a value
            query = """INSERT INTO "DimCommodity" (commodity_name) VALUES (%s)
                       ON CONFLICT (commodity_name) DO NOTHING;"""

            # Execute the query for multiple rows
            cursor.executemany(query, new_commo_values)

            # Commit the transaction
            conn.commit()

            print("New commodities inserted successfully.")

        except psycopg2.Error as e:
            print(f"An error occurred: {e}")

        finally:
            # Close the cursor and connection
            if cursor:
                cursor.close()
            if conn:
                conn.close()

        # Step 2: Transform & Load the fact table

        # Read dim tables from pgsql
        conn = connect_to_pgsql()
        print("Reading Market and Commodity Data from PostgreSQL.")

        read_market_sql_query = '''SELECT market_id, market_name FROM "DimMarket"'''
        read_commo_sql_query = '''SELECT commodity_id, commodity_name FROM "DimCommodity"'''

        dim_market_df = pd.read_sql_query(read_market_sql_query, conn)
        dim_commo_df = pd.read_sql_query(read_commo_sql_query, conn)

        #print(dim_market_df.head())

        # Change column names
        fact_df.columns = ['price_date', 'market_name', 'commodity_name', 'unit_of_measure', 
                           'price_range', 'max_price', 'min_price']

        # Change data types (max_price, min_price, price_date)
        fact_df['max_price']=fact_df['max_price'].astype('float')
        fact_df['min_price']=fact_df['min_price'].astype('float')
        fact_df['price_date'] = pd.to_datetime(fact_df['price_date'])

        # Merge the fact df and dim market df on the 'Market Name' column to replace name with id
        merged_df = fact_df.merge(dim_market_df, on='market_name', how='left')
        # Merge the fact df and dim market df on the 'Commodity Name' column to replace name with id
        merged_df = merged_df.merge(dim_commo_df, on='commodity_name', how='left')

        # Drop market name and commodity name
        final_fact_df = merged_df[['price_date', 'market_id', 'commodity_id', 'unit_of_measure', 
                                   'price_range', 'max_price', 'min_price']]

        #print(final_fact_df.head())


        # Step 3 - Insert Fact Table

        conn = connect_to_pgsql()
        print("Inserting Fact - Pricing Data.....")

        cursor = conn.cursor()

        try:
            # Insert statement template
            insert_query = """
                INSERT INTO "FactPricing" (price_date, market_id, commodity_id, unit_of_measure, price_range, max_price, min_price)
                VALUES (%s, %s, %s, %s, %s, %s, %s);
            """

            # Prepare data for batch insert using the DataFrame's values
            data_to_insert = final_fact_df.values.tolist()

            # Execute many for batch insert
            cursor.executemany(insert_query, data_to_insert)

            # Commit the transaction
            conn.commit()
            print("Pricing Data inserted successfully!")

        except Exception as e:
            # Rollback in case of an error
            if conn:
                conn.rollback()
            print(f"An error occurred: {e}")

        finally:
            # Close the cursor and connection
            if cursor:
                cursor.close()
            if conn:
                conn.close()
            print("Database connection closed.")

In [7]:
# Default URL of the website
url = "https://htwettoe.com/market?date="

# Create current month's dates list
current_date = datetime.today()

first_day = current_date.replace(day=1)
last_day = current_date.replace(day=calendar.monthrange(current_date.year, current_date.month)[1])

date_list = [first_day + timedelta(days=i) for i in range((last_day - first_day).days + 1)]

date_list_str = [date.strftime('%Y-%m-%d') for date in date_list]

# Create inserted date list
try:
    conn = connect_to_pgsql()
    query = 'SELECT DISTINCT price_date FROM "FactPricing"'
    pricing_date_df = pd.read_sql_query(query, conn)

    price_dates_lst = pricing_date_df['price_date'].values.tolist()
    price_dates_str_lst = [date.strftime('%Y-%m-%d') for date in price_dates_lst]
except Exception as e:
    print(f"There was an error while extracting price dates from DB: {e}")

date_list_str_to_insert = list(set(date_list_str) - set(price_dates_str_lst))
date_list_str_to_insert = sorted(date_list_str_to_insert)

for current_date in tqdm(date_list_str_to_insert):
    print(f"Start working on {current_date}.......")
    current_date_utc = datetime.strptime(current_date, "%Y-%m-%d")
    
    try:
        # Get the data
        print(url + current_date + "&")
        bsObj = connect_to_website(url + current_date + "&")
        # Extract data
        out_df = extract_data(bsObj, current_date)
        if out_df.empty:
            print(f"There is no data for this date: {current_date}\n")
            continue
        else:
            # Load data to PostgreSQL
            load_data_to_postgresql(out_df, current_date)
            # Wait for 1 second to prevent overloading the server
            time.sleep(5)
    except Exception as e:
        print(f"There was an error occurred: {e}")
        print(current_date)
        raise
        break

PostgreSQL Connection successful!


  0%|                                                                                           | 0/28 [00:00<?, ?it/s]

Start working on 2025-02-01.......
https://htwettoe.com/market?date=2025-02-01&


  4%|██▉                                                                                | 1/28 [00:00<00:18,  1.45it/s]

There is no data for this date: 2025-02-01

Start working on 2025-02-02.......
https://htwettoe.com/market?date=2025-02-02&


...

There is no data for this date: 2025-02-26

Start working on 2025-02-27.......
https://htwettoe.com/market?date=2025-02-27&


 96%|███████████████████████████████████████████████████████████████████████████████   | 27/28 [10:15<00:05,  5.51s/it]

There is no data for this date: 2025-02-27

Start working on 2025-02-28.......
https://htwettoe.com/market?date=2025-02-28&


100%|██████████████████████████████████████████████████████████████████████████████████| 28/28 [10:16<00:00, 22.02s/it]

There is no data for this date: 2025-02-28

